# Import libraries

In [1]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import numpy as np
from stock_prices_predictions.get_data import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras import layers
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import mean_absolute_percentage_error
from tensorflow.keras.models import load_model
from joblib import dump

# Looper

In [2]:
list_of_10_stocks = ['T', 'INTC', 'ADBE', 'JPM', 'PG', 'NVDA', 'AAPL', 'AMZN', 'UNH', 'MA']
#list_of_10_stocks = ['T']

In [3]:
for company in list_of_10_stocks:
    company = company
    days_ahead = 1
    
    #fetch data
    X_train, X_test = get_lstm_data(company)
    scaler = RobustScaler()
    scaler.fit(X_train)
    X_train_sc = scaler.transform(X_train)
    scaler_ = scaler
    #scale data
    X_test_sc = scaler.transform(X_test)
    X_train, y_train, y_train_simple = create_sequences_scaled_plus(pd.DataFrame(X_train_sc),pd.DataFrame(X_train), days_ahead)
    X_test, y_test, y_test_simple = create_sequences_scaled_plus(pd.DataFrame(X_test_sc), pd.DataFrame(X_test), days_ahead)
    
    #create model
    model = Sequential()
    model.add(LSTM(units=18, activation='tanh', input_shape= (19, 1)))
    #model.add(Dense(units=36, activation="tanh"))
    #model.add(Dense(units=10, activation="tanh"))
    model.add(Dense(units=1, activation="linear"))
    
    #fit model
    es = EarlyStopping(patience=20, restore_best_weights=True)
    model.compile(optimizer="nadam", loss="mse", metrics=["mae", "mape"])
    model.fit(X_train, y_train, epochs=200, batch_size=16, workers=-1, validation_split=0.2, callbacks=[es], verbose=0)

    # Export pipeline as pickle file  
    model.save(f'{company}_model.h5')
    dump(scaler, f'{company}_scaler.joblib')